In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import numpy as np
import pandas as pd

In [2]:
Gangwon = pd.read_csv('mini_project_1/Chung_buk.csv')
Gangwon

,Unnamed: 0,업종,업소명,주요품목,가격,연락처,주소
0,0,기타서비스업,청 판다,클렌징주스,3800원,010-2914-2231,"강원특별자치도 화천군 화천읍 상승로 60-6, 1층"
1,1,한식_기타,양양회포장,모둠회(소),39000원,01042361735,강원특별자치도 양양군 강원특별자치도 양양군 양양읍 남문7길 7-4
2,2,이미용업,은조헤어샵,커트(어르신),8000원,0507-1339-3560,강원특별자치도 원주시 반곡길 46
3,3,한식_일반,푸줏간정육식당,삼겹살(300g),16000원,033-345-0899,강원특별자치도 횡성군 강원특별자치도 횡성읍 화성로 24
4,4,한식_일반,강림산초두부,두부전골,8000원,033-342-3392,강원특별자치도 횡성군 강원특별자치도 강림면 주천강로 1132
...,...,...,...,...,...,...,...
592,592,이미용업,진희미용실,파마,20000원,033-643-2025,강원특별자치도 강릉시 율곡로 2755(노암동)
593,593,이미용업,금빗미용실,파마,15000원,033-643-4590,강원특별자치도 강릉시 중앙시장2길 56(성남동)
594,594,이미용업,황실헤어클럽,파마,20000원,033-647-2885,강원특별자치도 강릉시 중앙시장길 13(성남동)
595,595,한식_육류,대형생삼겹생선구이,삼겹살200g,9000원,033-651-7060,강원특별자치도 강릉시 성덕포남로 103(입암동)


In [3]:
Gangwon.drop('Unnamed: 0',axis = 1, inplace = True)

Gangwon['주소_구,군'] = Gangwon['주소'].apply(lambda x:x.split(' ')[0])

Gangwon['주소_구,군'] = Gangwon['주소'].apply(lambda x:x.replace('\xa0',' '))
Gangwon['keyword'] = Gangwon['주소_구,군']

Gangwon_address_list = Gangwon['keyword'].apply(lambda x:x.split(' '))

new_Gangwon_address_list = []

for i,item in enumerate(Gangwon_address_list):
    #print(ad_li[i][:2])
    address = item[0] + ' ' + item[1]
    new_Gangwon_address_list.append(address)
    
new_Gangwon_address_list

Gangwon['keyword'] = new_Gangwon_address_list

Gangwon['keyword'] = Gangwon['keyword'] + ' ' + Gangwon['업소명']

In [4]:
# 카카오맵 url 가져오기
url = 'https://map.kakao.com/'

# 크롬 드라이버 실행하기
driver = webdriver.Chrome()

# 카카오맵 url로 이동하기
driver.get(url)

score_list = []

for key in Gangwon['keyword'][:597]:
    # 검색창 클릭하기
    search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')

    # keyword의 key를 검색창에 넣기
    search.send_keys(key)

    # 엔터키로 이동하기
    search.send_keys(Keys.RETURN)

    time.sleep(2)

    # 웹 스크립트 받아오기
    html = driver.page_source

    # 웹스크립트에서 원하는 콘텐츠 추출하기 - parsing
    soup = BeautifulSoup(html, 'html.parser')
    
    # keyword별 평점 가져오기
    try:
        # 평점 path 가져오기
        score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
        
        # score_path를 split한 후 3번째 문자 추출하기
        score = soup.select(score_path)[0].text.split('\n')[3]
        
        # 평점을 빈 리스트 L에 추가하기
        score_list.append(score)
        
    except:
        score_list.append('별점 없음')
        
    search.clear()
    
score_list

Gangwon['평점'] = score_list
Gangwon.drop('주소_구,군',axis = 1, inplace = True)
Gangwon.drop('keyword',axis = 1, inplace = True)
Gangwon.to_csv('Gangwonscore.csv', encoding='utf-8-sig')

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
C:\Users\vvv\AppData\Local\Temp\ipykernel_18400\1521161927.py:14: SyntaxWarning: invalid escape sequence '\.'
  search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
C:\Users\vvv\AppData\Local\Temp\ipykernel_18400\1521161927.py:33: SyntaxWarning: invalid escape sequence '\.'
  score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
